# LaBSE

conda activate new_okr_LaBSE

In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 51.1 MB/s eta 0:00:00


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, util
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import faiss
import numpy as np

In [2]:
df = pd.read_csv('/content/cleaned_df.csv')

In [3]:
df.head()

,Unnamed: 0,page_url,image_url,title,description,genres,date
0,1,https://myshows.me/view/1583/,https://media.myshows.me/shows/760/9/ec/9ec875...,Чак Норрис и Карате Коммандос,Герой Чака сражается с верными подручными прот...,"Боевик,, Семейный",15.09.1986 — 19.09.1986
1,2,https://myshows.me/view/34639/,https://media.myshows.me/shows/760/3/95/395ce9...,Безработный роман,Сын-хи мечтает стать драматической писательниц...,"Комедия,, Романтика",05.10.2013 — 07.12.2013
2,3,https://myshows.me/view/1201/,https://media.myshows.me/shows/760/9/64/964415...,Первый поцелуй,Мао вернулась в Японию учится. Здесь она вновь...,"Романтика,, Аниме",06.10.2007 — 29.03.2008
3,4,https://myshows.me/view/71621/,https://media.myshows.me/shows/760/4/cd/4cdfff...,Объявлен мёртвым,"Преследуя опасного преступника, сотрудник МВД ...","Криминал,, Драма,, Детектив",06.02.2021 — 06.02.2021
4,5,https://myshows.me/view/43631/,https://media.myshows.me/shows/760/c/ce/ccec19...,Время выбрало нас,Зима 1942 года. Ивану Воронецкому в качестве у...,"Военный,, Драма",22.10.1979 — 26.10.1979


In [8]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch

# Загрузка LaBSE
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

# Загрузка DistilBERT
distilbert_tokenizer = AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased')
distilbert_model = AutoModel.from_pretrained('distilbert-base-multilingual-cased')

# Загрузка tiny2 (альтернатива ruTinyBERT)
tiny2_tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')
tiny2_model = AutoModel.from_pretrained('cointegrated/rubert-tiny2')

# Перенос моделей на GPU (если доступен)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
distilbert_model.to(device)
tiny2_model.to(device)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(83828, 312, padding_idx=0)
    (position_embeddings): Embedding(2048, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-2): 3 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [10]:
import numpy as np

# Функция для получения эмбеддингов батчами
def get_embeddings_batch(texts, model, tokenizer=None, batch_size=32, max_length=128):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        if tokenizer:
            inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
            inputs = {key: val.to(device) for key, val in inputs.items()}
            with torch.no_grad():
                outputs = model(**inputs)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        else:
            batch_embeddings = model.encode(batch)
        embeddings.extend(batch_embeddings)
    return np.array(embeddings)

# Преобразование описаний сериалов
descriptions = df['description'].tolist()

# LaBSE
labse_embeddings = get_embeddings_batch(descriptions, labse_model)

# DistilBERT
distilbert_embeddings = get_embeddings_batch(descriptions, distilbert_model, distilbert_tokenizer)

# tiny2
tiny2_embeddings = get_embeddings_batch(descriptions, tiny2_model, tiny2_tokenizer)

In [13]:
import faiss

# Создание индексов
def create_faiss_index(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

# LaBSE
labse_index = create_faiss_index(labse_embeddings)

# DistilBERT
distilbert_index = create_faiss_index(distilbert_embeddings)

# tiny2
tiny2_index = create_faiss_index(tiny2_embeddings)

In [14]:
def search_series(query, model, tokenizer=None, index=None, top_k=5):
    if tokenizer:
        inputs = tokenizer([query], return_tensors="pt", padding=True, truncation=True, max_length=128)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        query_embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    else:
        query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    results = df.iloc[indices[0]]
    return results

# Пример использования
query = "комедия про друзей"
labse_results = search_series(query, labse_model, index=labse_index)
distilbert_results = search_series(query, distilbert_model, distilbert_tokenizer, distilbert_index)
tiny2_results = search_series(query, tiny2_model, tiny2_tokenizer, tiny2_index)

print("LaBSE Results:", labse_results[['title', 'description']])
print("DistilBERT Results:", distilbert_results[['title', 'description']])
print("tiny2 Results:", tiny2_results[['title', 'description']])

LaBSE Results:                           title  \
5091       Мальчик по соседству   
482                      Друзья   
13753              Ты прекрасна   
5687                     Короче   
15143  Comic Strip представляет   

                                             description  
5091   Веб-дорама основана на комиксе, который расска...  
482    Культовый ситком о жизни шестерых друзей-сосед...  
13753  Трогательная семейная драма о любви, дружбе и ...  
5687   Короткие комедийные истории, рассказанные от п...  
15143  Серия самодостаточных телефильмов, роли в кото...  
DistilBERT Results:                           title  \
4712       Сексуальное убийство   
9843   Под несчастливой звездой   
13184   Поделись счастьем своим   
7184     Смешарики снимают кино   
1731               Миф о Сизифе   

                                             description  
4712   «Сексуальное убийство» — великолепная пародия ...  
9843   Сериал основан на одноименной книге Мелинды Та...  
13184  В осн

In [18]:
import time

def measure_time(func, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)  # Передаем именованные аргументы
    end_time = time.time()
    return result, end_time - start_time

In [19]:
# Пример использования
query = "комедия про друзей"

# Сравнение времени
labse_results, labse_time = measure_time(search_series, query, labse_model, index=labse_index)
distilbert_results, distilbert_time = measure_time(search_series, query, distilbert_model, tokenizer=distilbert_tokenizer, index=distilbert_index)
tiny2_results, tiny2_time = measure_time(search_series, query, tiny2_model, tokenizer=tiny2_tokenizer, index=tiny2_index)

print(f"LaBSE Time: {labse_time:.4f} sec")
print(f"DistilBERT Time: {distilbert_time:.4f} sec")
print(f"tiny2 Time: {tiny2_time:.4f} sec")

LaBSE Time: 0.0260 sec
DistilBERT Time: 0.0522 sec
tiny2 Time: 0.0497 sec


In [20]:
import pickle

# Сохранение данных
with open('labse_index.pkl', 'wb') as f:
    pickle.dump(labse_index, f)

with open('distilbert_index.pkl', 'wb') as f:
    pickle.dump(distilbert_index, f)

with open('tiny2_index.pkl', 'wb') as f:
    pickle.dump(tiny2_index, f)